# Savanna project - fire scar data exploration

In [184]:
%matplotlib inline
import matplotlib.pyplot as plt

import pathlib
import collections

import numpy as np

import fiona
import shapely
import odc.geo.geom
import odc.geo.xr
import rioxarray

In [169]:
fire_scar_dir = pathlib.Path("~/data/raw/fire_scar/Fire_scars_1989_2023").expanduser()
fire_scar_path = fire_scar_dir / "JW_aust2022faa.shp"

In [171]:
handle = fiona.open(fire_scar_path)

In [178]:
def geom_from_shape(entry, crs):
    
    if entry.geometry.type == "Polygon":
        (outer, *inners) = entry.geometry.coordinates
        geom = odc.geo.geom.polygon(
            outer,
            crs,
            *inners,
        )

    elif entry.geometry.type == "MultiPolygon":
        geom = odc.geo.geom.multipolygon(
            coords=entry.geometry.coordinates,
            crs=crs,
        )
    else:
        raise ValueError()

    return geom

In [179]:
polygons = [
    geom_from_shape(
        entry=entry,
        crs=handle.crs,
    )
    for entry in handle
]

In [183]:
len(polygons)

29180

In [186]:
odc.geo.xr.rasterize

<function odc.geo._xr_interop.rasterize(poly: odc.geo.geom.Geometry, how: Union[float, int, odc.geo.types.Resolution, odc.geo.geobox.GeoBox], *, value_inside: bool = True, all_touched: bool = False) -> xarray.core.dataarray.DataArray>